In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
# Import all required libraries. Few I have added down wherever it was required.
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt 
import plotly.express as px
import os
import cv2
from PIL import Image
import keras
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import tensorflow as tf
from tensorflow.keras import models, layers
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau
#from tensorflow.keras.applications import EfficientNetB0, Xception
from tensorflow.keras.applications import Xception
from tensorflow.keras.optimizers import Adam

In [ ]:
# First fetch train and test dataset in your notebook

input_dir = "../input/cassava-leaf-disease-classification"

train_images_path = os.path.join(input_dir,"train_images")
test_images_path = os.path.join(input_dir,'test_images')

In [ ]:
train_images_path

In [ ]:
train = pd.read_csv('../input/cassava-leaf-disease-classification/train.csv')
train.head(10)

In [ ]:
train.shape # Number of images in folder.

In [ ]:
train[train['image_id'].str.contains('1001742395')]

In [ ]:
image_list = train['image_id'].to_list()
label_list = train['label'].to_list()
len(image_list)

In [ ]:
for i in range(5):
    print(image_list[i],"   ",label_list[i])

In [ ]:
import json

with open("../input/cassava-leaf-disease-classification/label_num_to_disease_map.json") as f:
    class_mapping = json.load(f)

class_mapping2 ={int(k):v for k,v in class_mapping.items()}

class_mapping2

# Distribution of Diseases

In [ ]:
plt.figure(figsize=(8,5))

sns.set_style('whitegrid')

ax=sns.countplot(data=train, x='label', palette="Pastel1")


#  '0': 'Cassava Bacterial Blight (CBB)
#  '1': 'Cassava Brown Streak Disease (CBSD)
#  '2': 'Cassava Green Mottle (CGM)
#  '3': 'Cassava Mosaic Disease (CMD)
#  '4': 'Healthy

In [ ]:
train2 = train.copy()
train2.replace({"label": class_mapping2}, inplace=True)

pie_df = train2['label'].value_counts().reset_index()
pie_df.columns = ['label', 'count']
fig = px.pie(pie_df, values = 'count', names = 'label', hole=.3, color_discrete_sequence = px.colors.qualitative.Pastel1)
fig.show()

In [ ]:
def plot_samples(class_):
    
    print(f'Some Sample Images belonging to Class {class_mapping[f"{class_}"]}')
    
    sample_images = train[train.label == class_].sample(8)
    
    plt.rcParams["axes.grid"] = False

    fig,ax = plt.subplots(nrows=2,ncols=4,figsize=(20,12))

    for e,img in enumerate(sample_images.image_id):
        image_path = os.path.join(input_dir,f'train_images/{img}')
        image = cv2.imread(image_path)
        ax[e//4][e%4].imshow(image)
    
    plt.show()

In [ ]:
plot_samples(0) #Cassava Bacterial Blight

In [ ]:
plot_samples(1) #Cassava Brown Streak Disease

In [ ]:
plot_samples(2) #Cassava Green Mottle

In [ ]:
plot_samples(3) #Cassava Mosaic Disease

# Converting from BGR to RGB

In [ ]:
# Cassava Bacterial Blight (CBB) Samples¶
sample_images = train[train.label == 0].sample(5)
plt.figure(figsize=(35, 20))
for e,img in enumerate(sample_images.image_id):
    plt.subplot(1, 5, e + 1)
    img = cv2.imread(os.path.join(input_dir,f'train_images/{img}'))
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    plt.imshow(img)
    
plt.show()

In [ ]:
# Cassava Brown Streak Disease (CBSD) Samples
sample_images = train[train.label == 1].sample(5)
plt.figure(figsize=(35, 20))
for e,img in enumerate(sample_images.image_id):
    plt.subplot(1, 5, e + 1)
    img = cv2.imread(os.path.join(input_dir,f'train_images/{img}'))
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    plt.imshow(img)

plt.show()

In [ ]:
# Cassava Green Mottle (CGM) Samples
sample_images = train[train.label == 2].sample(5)
plt.figure(figsize=(35, 20))
for e,img in enumerate(sample_images.image_id):
    plt.subplot(1, 5, e + 1)
    img = cv2.imread(os.path.join(input_dir,f'train_images/{img}'))
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    plt.imshow(img)
    
plt.show()

In [ ]:
# Cassava Mosaic Disease (CMD) Samples
sample_images = train[train.label == 3].sample(5)
plt.figure(figsize=(35, 20))
for e,img in enumerate(sample_images.image_id):
    plt.subplot(1, 5, e + 1)
    img = cv2.imread(os.path.join(input_dir,f'train_images/{img}'))
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    plt.imshow(img)
    
plt.show()

In [ ]:
# Healty Samples
sample_images = train[train.label == 4].sample(5)
plt.figure(figsize=(35, 20))
for e,img in enumerate(sample_images.image_id):
    plt.subplot(1, 5, e + 1)
    img = cv2.imread(os.path.join(input_dir,f'train_images/{img}'))
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    plt.imshow(img)
    
plt.show()

In [ ]:
# Model Implemetation and Augmentation
BATCH_SIZE =8 #Mini-Batch Gradient Descent
STEPS_PER_EPOCH = len(train)*0.8 / BATCH_SIZE
VALIDATION_STEPS = len(train)*0.2 / BATCH_SIZE
EPOCHS = 20
TARGET_SIZE = 350

In [ ]:
train.label = train.label.astype('str')
from keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(
                    rotation_range = 40,
                    width_shift_range = 0.2,
                    height_shift_range = 0.2,
                    shear_range = 0.2,
                    zoom_range = 0.2,
                    horizontal_flip = True,
                    vertical_flip = True,
                    fill_mode = 'nearest')

train_generator = train_datagen.flow_from_dataframe(train,
                         directory = os.path.join('../input/cassava-leaf-disease-classification/train_images'),
                         subset = "training",
                         x_col = "image_id",
                         y_col = "label",
                         target_size = (TARGET_SIZE, TARGET_SIZE),
                         batch_size = BATCH_SIZE,
                         class_mode = "sparse",
                         shuffle= True)


validation_datagen = ImageDataGenerator(validation_split = 0.2)

validation_generator = validation_datagen.flow_from_dataframe(train,
                         directory = os.path.join('../input/cassava-leaf-disease-classification/train_images'),
                         subset = "validation",
                         x_col = "image_id",
                         y_col = "label",
                         target_size = (TARGET_SIZE, TARGET_SIZE),
                         batch_size = BATCH_SIZE,
                         class_mode = "sparse")

In [ ]:
img_path = os.path.join('../input/cassava-leaf-disease-classification/train_images/1003442061.jpg')
img = image.load_img(img_path, target_size = (TARGET_SIZE, TARGET_SIZE))
img_tensor = image.img_to_array(img)
img_tensor = np.expand_dims(img_tensor, axis = 0)
img_tensor /= 255.

plt.imshow(img_tensor[0])
plt.show()

In [ ]:
generator = train_datagen.flow_from_dataframe(train.iloc[17:18],
                         directory = os.path.join('../input/cassava-leaf-disease-classification/train_images'),
                         x_col = "image_id",
                         y_col = "label",
                         target_size = (TARGET_SIZE, TARGET_SIZE),
                         batch_size = BATCH_SIZE,
                         class_mode = "sparse")

aug_images = [generator[0][0][0]/255 for i in range(10)]
fig, axes = plt.subplots(2, 5, figsize = (20, 10))
axes = axes.flatten()
for img, ax in zip(aug_images, axes):
    ax.imshow(img)
plt.tight_layout()
plt.show()

In [ ]:
def create_model_old():
    conv_base = Xception(include_top=False, input_tensor=None,
    pooling=None, input_shape=(TARGET_SIZE, TARGET_SIZE, 3), classifier_activation='softmax')
                               
    model = conv_base.output
    model = layers.GlobalAveragePooling2D()(model)
    model = layers.Dense(5, activation = "softmax")(model)
    model = models.Model(conv_base.input, model)

    model.compile(optimizer = Adam(lr = 0.001),
                  loss = "sparse_categorical_crossentropy",
                  metrics = ["acc"])
    return model

In [ ]:
# Check for TensorFlow or Thieno
from keras import backend as K
img_width = 150
img_height = 150
if K.image_data_format() == 'channels_first':
    input_shape = (3, img_width, img_height)
else:
    input_shape = (img_width, img_height, 3)

In [ ]:
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Activation, Dropout, Flatten, Dense
from keras import backend as K



In [ ]:
def create_model_1():
    model = Sequential()
    # Conv2D : Two dimenstional convulational model.
    # 32 : Input for next layer
    # (3,3) convulonational windows size
    model.add(Conv2D(512, (3, 3), input_shape=(TARGET_SIZE, TARGET_SIZE, 3)))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))

    model.add(Conv2D(256, (3, 3)))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))

    model.add(Conv2D(128, (3, 3)))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))

    model.add(Flatten()) # Output convert into one dimension layer and will go to Dense layer
    #model.add(Dense(512))
    #model.add(Activation('relu'))
    #model.add(bias_regularizer('tf.keras.regularizers.L1L2(l1=0.01, l2=0.001)'))
    model.add(Dense(512, activation = 'relu', bias_regularizer=tf.keras.regularizers.L1L2(l1=0.01, l2=0.001)))
    model.add(Dropout(0.7))
    model.add(Dense(5))
    model.add(Activation('softmax'))
    return model

In [ ]:
model = create_model_1()
model.summary()

In [ ]:
#model = create_model()
#model = create_model_1()
#import keras
#from keras import optimizers
#model.compile(loss='binary_crossentropy', optimizer=keras.optimizers.Adam(lr=.0001), metrics=['accuracy'])
#model.summary()

In [ ]:
# model = keras.models.load_model('../input/xception-best-weights/Xception_best_weights.h5')

In [ ]:
# Submission
submission_file = pd.read_csv(os.path.join('../input/cassava-leaf-disease-classification/sample_submission.csv'))
submission_file

In [ ]:
preds = []

for image_id in submission_file.image_id:
    print(image_id)
    image = Image.open(os.path.join(f'../input/cassava-leaf-disease-classification/test_images/{image_id}'))
    image
    image = image.resize((TARGET_SIZE, TARGET_SIZE))
    image = np.expand_dims(image, axis = 0)
    model.predict(image)
    preds.append(np.argmax(model.predict(image)))

submission_file['label'] = preds
submission_file

In [ ]:
submission_file.to_csv('submission.csv', index = False)

In [ ]:
import os
TEST_DIR = '../input/cassava-leaf-disease-classification/test_images/'
test_images = os.listdir(TEST_DIR)
predictions = []

for image in test_images:
    img = Image.open(TEST_DIR + image)
    img = img.resize((TARGET_SIZE, TARGET_SIZE))
    img = np.expand_dims(img, axis=0)
    predictions.extend(model.predict(img).argmax(axis = 1))
predictions

In [ ]:
sub = pd.DataFrame({'image_id': test_images, 'label': predictions})
display(sub)